# Librerias necesarias para el funcionamiento

In [ ]:
!pip install --upgrade gdown
!python -m spacy download es_core_news_lg
import spacy
import es_core_news_lg
from collections import Counter
import re
import csv
import gdown

# Descargar Archivos

In [ ]:
#tóxico archivos csv

!gdown 1gmV34z9n5URFOHBbbWtP-eNLjLWdtXNE #toxico_2011.csv
!gdown 1YdSD8firXalyWeepu0hwuGCYo5EDj47v #toxico_2022.csv

In [ ]:
#unicornio archivos csv

!gdown 10YiOpKeUswK74jijk2Go34cLVfN8pNa2 #unicornio_2011.csv
!gdown 1RG_7b4IiKXR_bkH3KQTvzsajCyQmKuYh #unicornio_2022.csv

In [ ]:
#hibernar archivos csv

!gdown 1xboi6KBAuQx-Fe8iM0GVQ7i3BlNcfAMR #hibernar_2011.csv
!gdown 1pxrNqFVR4QDcu1VlUFdbnHeGyroDNqa9 #hibernar_2022.csv

# Leer los archivos

In [ ]:
def abrir_csv(file_path) -> str:
    try:
        with open(file_path, 'r', encoding='utf-8', newline='') as csvfile:
            reader = csv.reader(csvfile)
            # Ignoramos las primeras cinco lineas que solo dan información de sketch engine
            for _ in range(5):
                next(reader)
            # Solo queremos guardar los contextos, no hace falta el origen, etc.
            content = ' '.join([' '.join(row[1:]) for row in reader])

            # Eliminamos elementos de formato no necesarios para el estudio
            symbol_pattern = re.compile(r'[><|]|/s><s')
            content = re.sub(symbol_pattern, '', content)

            return content
    #En caso de error
    except FileNotFoundError:
        return f"El archivo '{file_path}' no existe."
    except Exception as e:
        return f"Ha ocurrido un error: {e}"

In [ ]:
#toxico

toxico_11_csv = "toxico_2011.csv"
toxico_22_csv = "toxico_2022.csv"
toxico_11 = abrir_csv(toxico_11_csv)
toxico_22 = abrir_csv(toxico_22_csv)
print(toxico_22[:1000]) #comprobar formato

In [ ]:
#unicornio

unicornio_11_csv = "unicornio_2011.csv"
unicornio_22_csv = "unicornio_2022.csv"
unicornio_11 = abrir_csv(unicornio_11_csv)
unicornio_22 = abrir_csv(unicornio_22_csv)
print(unicornio_11[:1000])

In [ ]:
#hibernar

hibernar_11_csv = "hibernar_2011.csv"
hibernar_22_csv = "hibernar_2022.csv"
hibernar_11 = abrir_csv(hibernar_11_csv)
hibernar_22 = abrir_csv(hibernar_22_csv)
print(hibernar_22[:1000])

# Palabras más comunes

In [ ]:
nlp = es_core_news_lg.load()
nlp.max_length = 3000000

#Esta función devuelve 50 palabras de todo tipo (excepto palabras vacías y puntuación) según su frecuencia junto al neologismo
def get_words(file):
    doc = nlp(file)

    lemmatized_words = [token.lemma_
                        for token in doc
                        if not token.is_stop and not token.is_punct]

    word_freq = Counter(lemmatized_words)
    common_words = word_freq.most_common(50)
    return common_words

#Muy similar a la anterior pero en este caso solo sustantivos
def get_nouns(file, avoid_word):
    doc = nlp(file)

    lemmatized_nouns = [token.lemma_
                        for token in doc
                        if (not token.is_stop and
                            not token.is_punct and
                            token.pos_ == "NOUN" and
                            token.lemma_ != avoid_word)]

    noun_freq = Counter(lemmatized_nouns)
    common_nouns = noun_freq.most_common(50)

    # Enumerate and print the results
    for index, (noun, frequency) in enumerate(common_nouns, start=1):
        print(f"{index}. {noun}: {frequency}")

In [ ]:
#Para cambiar de neologismo solo eliminar el término en las casilla
# a la derecha llamada neo
#Opciones: tóxico, unicornio, hibernar


neo = "hibernar" # @param {type:"string"}
if neo == "tóxico":
    corpus = "toxico"
elif neo == "hibernar" or neo == "unicornio":
    corpus = neo
else:
    print("Por favor, introduzca un neologismo válido.")

corpus_11 = str(corpus) + "_11"
corpus_22 = str(corpus) + "_22"
evitar_neologismo = neo

In [ ]:
# corpus 2011
print(f"Buscando los sustantivos más frecuentes en el contexto general de {evitar_neologismo} en 2011...")
sustantivos_11 = get_nouns(globals()[corpus_11], evitar_neologismo)
print(sustantivos_11)

In [ ]:
# corpus 2021-22
print(f"Buscando los sustantivos más frecuentes en el contexto general de {evitar_neologismo} en 2021-22...")
sustantivos_22 = get_nouns(globals()[corpus_22], evitar_neologismo)
print(sustantivos_22)

# Dependencia Sintáctica

In [ ]:
#Esta función coge el lemma del neologismo
def get_dep_sus(file, target_w):
    doc = nlp(file)

    target_word = target_w
    target_lemma = nlp(target_word)[0].lemma_

    linked_words = []

    for token in doc:
        if not token.is_stop and not token.is_punct and token.pos_ == "NOUN" and token.head.lemma_ == target_lemma:
            linked_words.append((token.lemma_, token.dep_))

    word_frequency = Counter(linked_words)

    print(f"Top 25 palabras relacionadas con el lemma'{target_word}': \n")
    for (word, dep), count in word_frequency.most_common(25):
        print(f"'{word}' funciona como {dep}: {count} veces")
        # Print example sentences for the word
        #print("Example sentences:")
        #example_sentence_counter = 0
        #for sent in doc.sents:
            #if word in [token.lemma_ for token in sent]:
               # print(sent.text)
                #example_sentence_counter += 1
                #if example_sentence_counter >= 2:  # Change to limit the number of example sentences per word
                   # break  # Stop after printing the specified number of example sentences
        #print()

def get_suj_neo(file, target_w):
    doc = nlp(file)

    target_word = target_w
    target_lemma = nlp(target_word)[0].lemma_

    linked_words = []
    for token in doc:
        if not token.is_stop and not token.is_punct and token.pos_ == "NOUN" and token.head.lemma_ == target_lemma and token.dep_ == "nsubj":
            linked_words.append((token.lemma_, token.dep_))
    word_frequency = Counter(linked_words)

    print(f"Top 25 palabras 'sujeto' en relacion con '{target_word}': \n")
    for (word, dep), count in word_frequency.most_common(25):
        print(f"'{word}' funciona como {dep}: {count} veces")

In [ ]:
neologismo = neo

In [ ]:
get_dep_sus(globals()[corpus_11], neologismo)

In [ ]:
#esta funcion sirve para comprobar solamente los sustantivos etiquetados con nsubj

#get_suj_neo(globals()[corpus_11], neologismo)

In [ ]:
get_dep_sus(globals()[corpus_22], neologismo)

In [ ]:
#get_suj_neo(globals()[corpus_22], neologismo)

In [ ]:
#se ha usado esta función para buscar contextos donde ocurran una palabra especifica y el neologismo
def checker(file, target_w1, target_w2):
    doc = nlp(file)

    target_lemma1 = nlp(target_w1)[0].lemma_
    target_lemma2 = nlp(target_w2)[0].lemma_

    linked_sentences = []

    for sent in doc.sents:
        word_found1 = False
        word_found2 = False
        for token in sent:
            if token.lemma_ == target_lemma1:
                word_found1 = True
            if token.lemma_ == target_lemma2:
                word_found2 = True
        if word_found1 and word_found2:
            linked_sentences.append(sent.text)

    print(f"Contextos que incluyen '{target_w1}' y '{target_w2}' :\n")
    for sentence in linked_sentences:
        print(sentence)


In [ ]:
palabra = "persona" #@param {type:"string"}
checker(globals()[corpus_22], neologismo, palabra)